# Deliverable 3

Trying to get more than 75% of accuracy

In [21]:
#!pip install tensorflow
#!pip install git+https://github.com/divamgupta/image-segmentation-keras.git

  Cloning https://github.com/divamgupta/image-segmentation-keras.git to /tmp/pip-req-build-mef5gprv
  Running command git clone -q https://github.com/divamgupta/image-segmentation-keras.git /tmp/pip-req-build-mef5gprv
  Resolved https://github.com/divamgupta/image-segmentation-keras.git to commit dc830bbd76371aaedbf8cb997bdedca388c544c4


In [1]:
# Import our dependencies
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler,OneHotEncoder
import pandas as pd
import tensorflow as tf

#  Import and read the charity_data.csv.
import pandas as pd 
application_df = pd.read_csv("charity_data.csv")

In [2]:
application_df = application_df.drop(['EIN'], 1)

In [3]:
atc = application_df.APPLICATION_TYPE.value_counts()

In [4]:
# Determine which values to replace if counts are less than ...?
replace_app = atc[atc < 500].index

# Replace in dataframe
for app in replace_app:
    application_df.APPLICATION_TYPE = application_df.APPLICATION_TYPE.replace(app, "Other")
    
# Check to make sure binning was successful
application_df.APPLICATION_TYPE.value_counts()

T3       27037
T4        1542
T6        1216
T5        1173
T19       1065
T8         737
T7         725
T10        528
Other      276
Name: APPLICATION_TYPE, dtype: int64

In [5]:
cv = application_df.CLASSIFICATION.value_counts()

In [6]:
# Determine which values to replace if counts are less than ..?
rc = list(cv[cv < 500].index)

# Replace in dataframe
for cls_ in rc:
    application_df.CLASSIFICATION = application_df.CLASSIFICATION.replace(cls_, "Other")

In [7]:
app_category = application_df.dtypes[application_df.dtypes == "object"].index

In [8]:
enc = OneHotEncoder(sparse=False)

# Fit and transform the OneHotEncoder using the categorical variable list
encodedf = pd.DataFrame(enc.fit_transform(application_df[app_category]))

# Add the encoded variable names to the dataframe
encodedf.columns = enc.get_feature_names(app_category)

# Merge one-hot encoded features and drop the originals
application_df = application_df.merge(encodedf, left_index=True, right_index=True)
application_df = application_df.drop(columns = app_category)
# Split our preprocessed data into our features and target arrays
t1 = application_df["IS_SUCCESSFUL"].values
t2 = application_df.drop(["IS_SUCCESSFUL"],1).values

# Split the preprocessed data into a training and testing dataset

# Split the preprocessed data into a training and testing dataset
X_train, X_test, y_train, y_test = train_test_split(t2, t1, random_state=78)

# Create a StandardScaler instances
scaler = StandardScaler()

# Fit the StandardScaler
X_scaler = scaler.fit(X_train)

# Scale the data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

In [9]:
# Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.
#  YOUR CODE GOES HERE
number_input_features = len(X_train[0])
hidden_nodes_l1 =  80
hidden_nodes_l2 = 30
hidden_nodes_l3 = 20

nn = tf.keras.models.Sequential()

# First hidden layer
nn.add(
    tf.keras.layers.Dense(units=hidden_nodes_l1, 
                          input_dim=number_input_features, 
                          activation="relu")
    )

# Second hidden layer
nn.add(
    tf.keras.layers.Dense(units=hidden_nodes_l2, 
                          activation="sigmoid"))

# Output layer
nn.add(
    tf.keras.layers.Dense(units=1, 
                          activation="sigmoid"))

# Check the structure of the model
nn.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 80)                1569040   
_________________________________________________________________
dense_1 (Dense)              (None, 30)                2430      
_________________________________________________________________
dense_2 (Dense)              (None, 1)                 31        
Total params: 1,571,501
Trainable params: 1,571,501
Non-trainable params: 0
_________________________________________________________________


In [10]:
from tensorflow.keras.callbacks import ModelCheckpoint

checkpoint_path = "weights.{epoch:02d}.hdf5"

# Compile the model
nn.compile(loss="binary_crossentropy", 
           optimizer="adam", 
           metrics=["accuracy"])

# Create a callback that saves the model's weights every 5 epochs
cp_callback = ModelCheckpoint(
    filepath=checkpoint_path,
    verbose=1,
    save_weights_only=True,
    save_freq=1000)

# Train the model
fit_model = nn.fit(X_train_scaled,
                   y_train,epochs=30,
                   callbacks=[cp_callback])

# Evaluate the model using the test data
model_loss, model_accuracy = nn.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

[2021-09-20 09:48:14.359 ip-172-16-54-72:34397 INFO utils.py:27] RULE_JOB_STOP_SIGNAL_FILENAME: None
[2021-09-20 09:48:14.656 ip-172-16-54-72:34397 INFO profiler_config_parser.py:111] Unable to find config at /opt/ml/input/config/profilerconfig.json. Profiler is disabled.
Train on 25724 samples
Epoch 1/30
  576/25724 [..............................] - ETA: 2:11 - loss: 0.6493 - accuracy: 0.6458 
Epoch 00001: saving model to weights.01.hdf5
 1664/25724 [>.............................] - ETA: 45s - loss: 0.6147 - accuracy: 0.6683 
Epoch 00001: saving model to weights.01.hdf5
 2688/25724 [==>...........................] - ETA: 27s - loss: 0.5981 - accuracy: 0.6920
Epoch 00001: saving model to weights.01.hdf5
 3680/25724 [===>..........................] - ETA: 19s - loss: 0.5734 - accuracy: 0.7144
Epoch 00001: saving model to weights.01.hdf5
 4736/25724 [====>.........................] - ETA: 15s - loss: 0.5544 - accuracy: 0.7261
Epoch 00001: saving model to weights.01.hdf5
 5888/25724 [==